In [354]:
import os
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import func4behav as fb
from scipy.stats import lognorm, exponnorm, invgauss
from scipy.optimize import curve_fit
from sklearn.metrics import mean_squared_error
from pyddm import Model, Sample, Fittable, Fitted
from pyddm.models import DriftConstant, NoiseConstant, BoundConstant, OverlayNonDecision, ICPointSourceCenter, LossRobustLikelihood
from pyddm.functions import fit_adjust_model, display_model
import pyddm.plot

import imp
imp.reload(fb)

<module 'func4behav' from 'c:\\Users\\Tianyi Zheng\\OneDrive - neuron.t.u-tokyo.ac.jp\\Documents\\zheng\\mywork\\attention_tES\\tes-attention\\processing\\behavior\\func4behav.py'>

In [355]:
def distribution_param(case_list, cutoff):

    _, expt_info = fb.create_allsubs_compare()
    param_record = pd.DataFrame(columns=['subject id', 'case', 'group', 'session', 'method', 'params', 'log likelihood', 'AIC'])
    
    session_name = ['before', 'after']
    methods_name = ['shift lognorm', 'exgaussian', 'ddm', 'shift invgauss']
    methods = [fb.fit_shift_lognorm, fb.fit_exgaussian, fb.fit_ddm, fb.fit_shift_invgauss]

    # Loops
    record_idx = 0
    for subject_id in range(1,19):
        print("Subject id:" + str(subject_id) + " is processing...")
        real_or_sham = expt_info.loc[expt_info['subject id'] == subject_id, 'Real stimulation'].values[0]

        for case in case_list:
            # print("Case:" + str(case) + " is processing...")
            rt_before, rt_after = make_rt_data(subject_id, case, cutoff)

            for method_idx, method in enumerate(methods):
                for session_idx, rt in enumerate([rt_before, rt_after]):
                    param, log_L, AIC = method(rt, cutoff)

                    param_record.loc[record_idx, 'subject id'] = subject_id
                    param_record.loc[record_idx, 'case'] = case
                    param_record.loc[record_idx, 'group'] = "real" if real_or_sham == 1 else "sham"
                    param_record.loc[record_idx, 'session'] = session_name[session_idx]
                    param_record.loc[record_idx, 'method'] = methods_name[method_idx]
                    param_record.loc[record_idx, 'params'] = param
                    param_record.loc[record_idx, 'log likelihood'] = log_L
                    param_record.loc[record_idx, 'AIC'] = AIC

                    record_idx += 1
                    
    return param_record


def make_rt_data(subject_id, case, cutoff):
    
    behavior_before, behavior_after = fb.load_behavior(subject_id)
    behavior_before, behavior_after = fb.filter_behav(case, behavior_before, behavior_after)
    rt_before = behavior_before.loc[(behavior_before['response'] == 1) & (behavior_before['reaction time'] > 0.05) & (behavior_before['reaction time'] < cutoff)]
    rt_after = behavior_after.loc[(behavior_after['response'] == 1) & (behavior_after['reaction time'] > 0.05) & (behavior_after['reaction time'] < cutoff)]

    rt_before = rt_before['reaction time'].values
    rt_after = rt_after['reaction time'].values

    return rt_before, rt_after

In [356]:
all_real = [1, 3, 4, 5, 9, 12, 13, 17, 18]
all_sham = [2, 6, 7, 8, 10, 11, 14, 15, 16]
case_list = ['all', 'endo', 'exo', 'valid', 'endo valid', 'exo valid', 'invalid', 'endo invalid', 'exo invalid', 'stim left', 'endo stim left', 'endo stim right', 'stim right', 'exo stim left', 'exo stim right']
cutoff = 1
# param_record = distribution_param(case_list, cutoff)
# param_record.to_csv('../../../docs/report/data/param_record.csv', index=False)

## compute P value

In [357]:
param_record_path = os.path.join('..', '..', '..', 'docs', 'report', 'data', 'param_record.csv')
param_record = pd.read_csv(param_record_path)
param_record

,subject id,case,group,session,method,params,log likelihood,AIC
0,1,all,real,before,shift lognorm,"[0.530450629232766, 0.09696554056594511, 0.119...",184.279441,-362.558881
1,1,all,real,after,shift lognorm,"[0.1727647541500616, 0.3451221542342069, -0.15...",162.489464,-318.978928
2,1,all,real,before,exgaussian,"[2.9564068064090367, 0.1736784921094102, 0.019...",185.574583,-365.149166
3,1,all,real,after,exgaussian,"[1.1293728835241597, 0.1461682210666338, 0.040...",164.208207,-322.416415
4,1,all,real,before,ddm,"[9.69515774149879, 1.7652210978910852, 1.16170...",182.518774,-357.037547
...,...,...,...,...,...,...,...,...
2155,18,exo stim right,real,after,exgaussian,"[4.223375125196021, 0.24303794121277486, 0.014...",48.557894,-91.115789
2156,18,exo stim right,real,before,ddm,"[8.798864077268108, 1.4167599334854657, 2.0381...",36.628143,-65.256286
2157,18,exo stim right,real,after,ddm,"[9.961070584448608, 1.8745560474589342, 0.9924...",48.672956,-89.345912
2158,18,exo stim right,real,before,shift invgauss,"[0.11336152603077676, 0.10825551392430313, 2.0...",36.628786,-67.257573


In [358]:
df = param_record.loc[(param_record['subject id'] == 5) & (param_record['case'] == 'all')]
df = df.loc[df['session'] == 'after', 'params']
print(df)
first_values = [float(x.split(',')[0].strip('[')) for x in df.astype(str)]
first_values

481    [0.45387700889482596, 0.10626482339761094, 0.1...
483    [2.2924812717124716, 0.20530836712574724, 0.02...
485    [9.230828876358776, 1.5242298376376473, 1.0788...
487    [0.20990740594982063, 0.1351806400869016, 0.57...
Name: params, dtype: object


[0.45387700889482596,
 2.2924812717124716,
 9.230828876358776,
 0.20990740594982063]

In [359]:
def compute_p(data1, data2, p_method):
    if p_method == 1:
        _, p_value = stats.mannwhitneyu(data1, data2)
    else:
        _, p_value = stats.ttest_ind(data1, data2)
    return p_value


def get_p_values(df, param_number):
    p_values = np.ndarray([])
    param_before = df.loc[df['session'] == 'before', 'params']
    param_after = df.loc[df['session'] == 'after', 'params']
    for i in range(param_number):
        before = [float(x.split(',')[i].strip('[]')) for x in param_before.astype(str)]
        after = [float(x.split(',')[i].strip('[]')) for x in param_after.astype(str)]
        p_values = np.append(p_values, compute_p(before, after, 1))
        
    return p_values

In [360]:
methods_name = ['shift lognorm', 'exgaussian', 'ddm', 'shift invgauss']
method_params = [3,3,4,3]
p_table = pd.DataFrame(columns=['case', methods_name[0] + ': sham', methods_name[0] + ': real', methods_name[1] + ': sham', methods_name[1] + ': real',
                                 methods_name[2] + ': sham', methods_name[2] + ': real', methods_name[3] + ': sham', methods_name[3] + ': real'])
p_table['case'] = case_list


In [361]:
for case in case_list:
    df_case = param_record.loc[param_record['case'] == case]
    for group in ['sham', 'real']:
        df_group = df_case.loc[df_case['group'] == group]
        for i, method in enumerate(methods_name):
            df_method = df_group.loc[df_group['method'] == method]
            p_values = get_p_values(df_method, method_params[i])

            p_table.loc[p_table['case'] == case, method+': '+group] = p_values

[       nan 0.85981921 0.72393204 0.92963652]


ValueError: Must have equal len keys and value when setting with an iterable

In [ ]:
p_table

,case,shift lognorm: sham,shift lognorm: real,exgaussian: sham,exgaussian: real,ddm: sham,ddm: real,shift invgauss: sham,shift invgauss: real
0,all,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,endo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,exo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,valid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,endo valid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,exo valid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,invalid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,endo invalid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,exo invalid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,stim left,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
